## ALASI 2017 Workshop: An Introduction to Text Analysis for Learning Analytics

#### Organisers: Antonette Shibani, Sophie Abel & Andrew Gibson  (UTS)

In this workshop, we demonstrate how Natural Language Processing (NLP) technologies can be used to help us computationally process large scale of textual data, and we examine how these kinds of technologies can play a positive role in learning contexts.The entire session will be hands-on, interacting with a text analytics system via this Jupyter notebook which runs in Python. 

---

### The Writing Analytics Process

Using some basic writing analytics, *this notebook demonstrates the process through which pedagogy and analytics come together* in order to provide formative feedback to students on their writing.

The key elements of the process are:

- A clearly defined learning task
- An understanding of the students response to the task (the writing)
- One or more hypotheses on how aspects of the learning task may be related to computationally identifiable features in the student writing

Throughout the process, the aim is to balance what is possible with the technology (i.e. Natural Language Processing - NLP) with what is valuable in terms of student learning. This is illustrated in the following diagram:

<img src="WAlayers.png" style="width: 60%;"/>




---

### Writing Task Definition

**Aims:** This assessment aims to develop your skills in building an argument and providing evidence to support your claims.

**Task:** You will write an argumentative essay on the ethical issues arising from our treatment of animals. You will consider what rights animals should be granted and what ethical implications arise between differentiating people from animals. Your discussion should include philosophical thinking on animal ethics.

In this assignment, you will: 
*	Identify an ethical issue 
*	Locate 4 recent articles relevant to the issue of a sufficiently high quality to support an academic discussion 
* Link your discussion to 2 philosophers from this list: Tom Regan, Immanuel Kant, René Descartes, Thomas Aquinas, Aristotle, Peter Carruthers, Joel Feinberg, Raymond Frey, Jeremy Bentham, Peter Singer


**Assessment:** The criteria on which your essay will be assessed is:

|             Assessment Criteria            |
| -------------        | ------------------ |
| **Structure**        | *Clear introduction (Background, Thesis statement, discussion topics)* |
|                      | *Selection of appropriate texts* |
|                      | *Clear synthesis of main points* |
|                      | *Organization of content supports argument* |
|                      | *Suitable conclusion* |
|**Written Expression**| *Academic writing style (third person)*  |
|                      | *Correct English grammar, spelling and punctuation* |
|                      | *Limit of 350 words* |
| **Referencing**      | *Correct Harvard citation in text*     |
|                      | *Correct Harvard reference list* |


---

### Student Response to Task

For the above writing task, consider that a student submits an essay to receive formative feedback. We start the analytics process by loading the student writing ready for analysis.

In [146]:
#Open the file and read it into a variable 'text'
file = open("sampleText.txt")
rawtext = file.read()
file.close()

#Split the document by newlines to give us paragraphs
doc = rawtext.split("\n")
print(doc)

['Today most of us accept science. Science of today, especially Darwins Evolutionary Theory teaches us that the human race has developed from animals and that humanbeings basically are animals among other animals. Eventhough we do agree with this aspect of science we tend to consider man as the almighty, powerful, crown of creation with a natural right to treat animals as we please and use them for our own purposes as food, tools and experiment instruments, and I belive this is wrong and in this essay I will explain why we morally have no right to exploit animals. I am also going to show how meaningless many of the experiments made on animals are.', "To begin with I would like to say that the line we draw today in order to distinguish ourselves from animals is strictly arbitrary, illogical and selfish and there isn't a quality and that includes all humanity that empowers us a narural position as top of the food chain and superiour to animals. Most humanbeings have unique capacities tho

However, this is not very human friendly output. A better display might be to use the HTML of the browser. To do this, we need to load some additional python libraries.

In [147]:
#Libraries contain functions that we can use to perform complex tasks without rewriting many lines of code
from IPython.core.display import display, HTML  # Allows us to create annotated text using HTML and CSS

#A function to wrap text in a paragraph tag
def pTag(text):
    return "<p>"+text+"</p>"

#Wrap each paragraph of the doc for HTML display
paras = map(pTag,doc)
htmlDoc = HTML(''.join(paras))

display(htmlDoc)

> ** DISCUSSION **
>
>  - What types of feedback can we give to improve this essay? 
>  - Can we try to automate this feedback (on certain criteria) using textual features?

---

### Langage features in the writing

Let's now identify language features to provide useful feedback on the given essay. This is a ground-up approach where we first look at the pedagogial context to identify the scope of automated feedback and then bring in the analytics where it fits in.

Some example features that we can use to provide feedback in this context are below:
* Structural Metrics (sentence, word level)
* Vocabulary
* Mechanics (spelling, grammar)
* Rhetorical structures
* Referencing formats


---

### Formulating and refining hypotheses

**Writing Anaytics involves formulating hypothesis around how computation and pedagogy might connect in a way that provides practical learning benefits for the student.**

Each hypothesis should specific and testable. That is, the practical results of it working should be clearly identifiable. They are not lofty ideals or goals - if they test true then there should be real practical learning benefits for the student. 

In this example, we will focus on how simple features in the text may be computationally identifiable and turned into formative feedback for the student. We make use of our context knowledge, theory and text observations to formulate and refine these hypothesis. 

Example hypotheses: 
1. Feedback on the readability could be created from the detection of long sentences.
2. Feedback on referencing requirements of the task could be generated from the presence of names in the text.
3. Feedback on writing as third person could generated from the idenfication of first person pronouns. 
4. Feedback on wordiness could be based on the presence of too many adjectives.

For each hypothesis, code is written to analyse the text and create formative feedback for the student based on the analytics. In a complete scenario, the students reaction to this feedback would be analysed providing an evaluation of the hypothesis. On the basis of this evaluation, the hypothesis could be refined, or other aspects of the process could be adjusted to improve the practical outcomes for the student.

---

## Implementation

### Hypothesis 1 - feedback on readability

We will make use of the open-source API called [Text Analytics Pipeline (TAP)](https://github.com/uts-cic/tap) to receive analytics about the text. To use TAP's analytics capability, we need to query the TAP graphql endpoint with the text that we want analysed. To do this, we need to construct the ```query``` , and a ```request header``` to tell TAP about the data format. Requesting for and receiving analytics from TAP consists of the following steps:

**[1]** Write a function to get json data from TAP based on a given query

In [148]:
import json                                 # We need to be able to work whith JSON
from urllib import request, response        # To create requests to TAP and handle responses from TAP
import string                               # To help with cleaning text and visualising analytics

def getJsonFromTAP(query,text,url):
    variables = {'input': text}
    escapedQuery = query.replace("\n", "\\n") #query.encode('utf8').decode('unicode_escape')
    fullQuery = json.dumps({'query': escapedQuery, 'variables': variables})
    jsonHeader = {'Content-Type':'application/json'}
    tapReq = request.Request(url, data = fullQuery.encode('utf8'), headers = jsonHeader)
    tapResponse = ""
    try:
        tapResponse = request.urlopen(tapReq)
        body = tapResponse.read().decode('utf8')           
        return json.loads(body)
    except Exception as e:
        print(e)
        return json.dumps({})
    

**[2]** Create the query and get analytics from TAP

In [149]:
# A Query for sentence level metrics query for TAP with given text
metricsQuery = "query Metrics($input: String!){ metrics(text: $input) { analytics {words, sentences, sentWordCounts, averageSentWordCount } } }"

tapUrl = "http://tap-test.utscic.edu.au/"   # TAP URL
endpoint = "graphql"                        # The query endpoint on TAP
completeUrl = tapUrl + endpoint             # The complete url that the request is posted to

# Send the query to TAP along with the text to be analysed
jsonData = getJsonFromTAP(metricsQuery,rawtext,completeUrl)
print("Output json object from TAP:\n")
print(jsonData)

Output json object from TAP:

{'data': {'metrics': {'analytics': {'words': 378, 'sentences': 11, 'sentWordCounts': [6, 26, 66, 16, 54, 46, 17, 31, 23, 29, 64], 'averageSentWordCount': 34.36363636363637}}}}


**[5]** Extract some general sentence-level metrics to provide summary statistics, and sentence word counts to provide specific feedback.

In [150]:
# Get the metrics from the returned JSON
wordCount = jsonData.get('data').get('metrics').get('analytics').get('words') 
sentCount = jsonData.get('data').get('metrics').get('analytics').get('sentences')
avgSentWordCount = jsonData.get('data').get('metrics').get('analytics').get('averageSentWordCount')
avgSentWordCount = round(avgSentWordCount,2)
sentWordCounts = jsonData.get('data').get('metrics').get('analytics').get('sentWordCounts')

# Get the sentence length standard deviation
import statistics as stats
stddev = stats.stdev(sentWordCounts)

# A threshold for long sentences
threshold = int(avgSentWordCount)+int(stddev)

def longSents(sentCounts,threshold):
    return [i+1 for i,x in enumerate(sentCounts) if x >= threshold]

longSentences = longSents(sentWordCounts,threshold)

**[6]** Display the results

Teachers may be interested in basic metrics associated with student texts

In [151]:
#Defining formatting options for the text
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

#print(color.BOLD+"Your text:\n"+color.END,ta.value,"\n")
print(color.RED +"<<<<< SUMMARY STATS FOR TEACHER >>>>>\n"+color.END)

## General summary statistics of the text
#print(color.BOLD+"Summary Statistics on your text:"+color.END)

print ("Total Words: ",wordCount)
print ("Total Sentences: ",sentCount)
print ("Average Words in a sentence: ",avgSentWordCount)
print ("Long sentence threshold: ",threshold)
print ("Long sentences: ",longSentences)

<<<<< SUMMARY STATS FOR TEACHER >>>>>

Total Words:  378
Total Sentences:  11
Average Words in a sentence:  34.36
Long sentence threshold:  54
Long sentences:  [3, 5, 11]


For student facing feedback, we need to consider what may be helpful to the student. For these simple metrics, the total word count could be useful (if a length was set for the assignment), and making the student aware of long sentences may be helpful.

Adapting the paragraph html display above, we can highlight long sentences.


In [152]:
# TAP can return the sentences from the text...

#A function to wrap text in a span tag with 'long' class
def longTag(kv):
    idx = kv[0]+1
    text = kv[1]
    if idx in longSentences:
        return '<span class="long">'+text+'</span>'
    else: 
        return text
    
def markupParaSentences(para):
    sentencesQuery = "query Sentences($input: String!){ annotations(text: $input) { analytics {idx, original, tokens { term }} }}"
    jsonData = getJsonFromTAP(sentencesQuery,para,completeUrl)
    sentencesJson = jsonData.get('data').get('annotations').get('analytics') #.get('original') 
    def getSentence(json):
        return (json.get('idx'),json.get('original'))
    sentences = map(getSentence,sentencesJson)
    pText =  ' '.join(map(longTag,sentences))
    return '<p>'+pText+'</p>'

comment = """
<h4>Feedback</h4>
<p class="feedback"><b>Long Sentences - </b>When you write, consider making it easy for the reader. Sentences that are
too long can be difficult to read. They can cause the reader to re-read the sentence multiple times to try and 
understand it, interrupting the flow of the text. Sentences that may be too long are <span class="long">highlighted 
in yellow</span>. Consider breaking these into smaller sentences to help make your writing easier to read.</p>

<p><b>Your text:</b></p>
"""
html = HTML(comment + ''.join(map(markupParaSentences,doc)))

css = HTML("""
<style>
.feedback {
    color: darkslateblue;
}
.long {
    background-color: yellow;
}
</style>

""")

display(css,html)

> ** DISCUSSION **
>
>  - Is this writing feedback applicable for all kinds of writing?
>  - Can the feedback be better expressed?
>  - Can we come up with a statistically valid threshold for the maximum words allowed in a sentence?


---

### Hypothesis 2 - feedback on refering to sources

This hypothesis is based around linking the identification of Named Entities (People, Organisations, Locations, Dates) to the requirement in the learning task to refer to at least 2 philosophers in a given list. By extracting names, we should be able to check this requirement.

In [153]:
#Extracting named entities from the input text using NER

# function to extract the tags
def getPosNer(json):
    return(json.get('term'),json.get('nertag'),json.get('postag'))
    
# function to get a list of tokens from a sentence
def getTokens(json):
    tokens = json.get('tokens')
    return list(map(getPosNer,tokens))

# the main function that initiates the TAP query and extracts the results
def getTags(para):
    query = "query TokeniseWithNer($input: String!){  annotations(text:$input,pipetype:\"ner\") { analytics {tokens {term,postag, nertag} } }}"
    jsonData = getJsonFromTAP(query,para,completeUrl)
    annotationsJson = jsonData.get('data').get('annotations').get('analytics') #.get('original')
    return list(map(getTokens,annotationsJson))

# function to turn a nested list into a list of strings
def flatten(nestList):
    newList = []
    for lst in nestList:
        for tups in lst:
            newList = newList + tups
    return newList

# a filter for named entities
def isNE(tagTuple):
    return tagTuple[1] != 'O'

# extract terms from the tuple
def getTerms(tagTuple):
    return tagTuple[0]


# put all of the above functions together...

# get the tags
tags = flatten(map(getTags,doc))
#print(tags)

# get the named entities
names = list(map(getTerms,filter(isNE,tags)))
print(names)

['Darwins', 'Evolutionary', 'Theory', 'Carruthers']


The named entities found in the student's text need to be cross checked with those required by the task. For the purposes of this example, the task names are manually entered into the `definednames` list.

In [154]:
definednames = ['Regan', 'Kant', 'Descartes', 'Aquinas', 'Aristotle', 'Carruthers', 'Feinberg', 'Frey', 'Bentham', 'Singer']

#Find the list of names also present in our defined list of authors
included = list(set(definednames).intersection(set(names)))
print("Included philosophers: ",included)

#Find the list of names missing in the input text
missing = list(set(definednames)-set(names))
print("Missing philosophers: ",missing)


Included philosophers:  ['Carruthers']
Missing philosophers:  ['Aristotle', 'Aquinas', 'Singer', 'Descartes', 'Frey', 'Kant', 'Bentham', 'Feinberg', 'Regan']


The task required that a minimum of 2 philosophers were included, so we generate a feedback message along these lines based on the analysis results. 

In [160]:
def referenceFeedback(included,missing,threshold):
    header = """<h4>Feedback</h4>
    <p class="feedback"><b>Refering to authoritive sources - </b>It is important to refer to authoritative sources
    in your writing.</p>"""

    negFeedback =  """<p class="feedback">The assessment task requires that you refer to a minimum number of authoritative sources,
    yet it appears that you have <b>not</b> met this requirement. Check the list of missing sources and consider 
    how you might modify your text.</p>"""

    posFeedback = """<p class="feedback">It appears that you have met the task requirement to refer to a minimum number of sources.</p>"""
    
    footer_incl = """<p><span class="feedback">Sources included: </span>
        <span class="included">"""+ ', '.join(included) + "</span></p>"
    footer_miss = """<p><span class="feedback">Sources not included: </span>
        <span class="missing">"""+ ', '.join(missing) + "</span></p>"

    if(len(included)<threshold):
        return header + negFeedback + footer_incl + footer_miss
    else:
        return header + posFeedback + footer_incl


css = HTML("""
<style>
.included {
    color: green;
}
.missing {
    color: red;
}
</style>

""")

feedback = HTML(referenceFeedback(included,missing,2))
display(css,feedback)

> ** DISCUSSION **
>
>  - Can we define list of author names automatically?
>  - Can we extract them from the given assessment task/ criteria for scalablility?


---

### Hypotheses 3 & 4 - feedback on first person and wordiness

Next, we are going to generate writing feedback based on the parts-of-speech (pos) used in the text. We'll make use of hypotheses 3 and 4 to analyse text by identifying `personal pronouns` and `adjectives` present in the text. We hypothesize that good academic writing should not contain personal pronouns to ensure formal language. We also hypothesize that if a text contains many adjectives, it leads to wordy text with less clarity.

In [156]:
#Looking for personal pronoun usage in the text
def isFirstPerson(tagTuple):
    return tagTuple[2].startswith('PRP') and tagTuple[0].lower() not in ['they','their','it','them']

personal = set(map(getTerms,filter(isFirstPerson,tags)))
print(personal)

{'we', 'I', 'We', 'our', 'ourselves', 'us'}


In [157]:
#Looking for qualifier term usage in the text (adjectives, adverbs)
def isQualifier(tagTuple):
    return tagTuple[2].startswith('JJ') or tagTuple[2].startswith('RB') 

qualifier = set(map(getTerms,filter(isQualifier,tags)))
print(qualifier)
#qualterms =[]
#myposlist2 = ['RB','RBR','RBS','JJ','JJR','JJS']
#myposlist2 = ['JJ','JJR','JJS']
#for key, value in posdict.items():
#    if value in myposlist2:
#        qualterms.append(key)
        
#print(qualterms)

{'important', 'just', 'human', 'such', 'different', 'often', 'logical', 'own', 'morally', "n't", 'natural', 'mental', 'illogical', 'also', 'personally', 'selfish', 'especially', 'very', 'most', 'strictly', 'same', 'narural', 'wrong', 'useful', 'Thus', 'unique', 'meaningless', 'Most', 'arbitrary', 'psychiatric', 'not', 'intellectual', 'powerful', 'scientifically', 'though', 'wide', 'other', 'higher', 'bad', 'basically', 'psycologically', 'intelectual', 'good', 'more', 'conscious', 'many'}


To provide writing feedback, let's also annotate the given text this time along with the feedback comments. This can help direct students' attention to certain parts of essay for improvements.

In [158]:
import re

def markupTerms(text,terms,classname):
    for term in terms:
        s = {}
        s[0] = term
        s[1] = '<span class="'+classname+'">'+term+'</span>'
        text = re.sub(r"\b%s\b" % s[0] , s[1], text)
    return text

def markupFirstPerson(text):
    return markupTerms(text,personal,"firstperson")

def markupQualifiers(text):
    return markupTerms(text,qualifier,"qualifier")

def wrapPara(text):
    return '<p>'+text+'</p>'
    
comment = """
<h4>Feedback</h4>
<p class="feedback"><b>Third person language - </b>This task requires that you write from a third person point of view
which means that you should avoid referring to yourself. You have used a number of first person pronouns in your writing,
and these should be avoided. For example, instead of saying "I agree with position X", you may say something like "X is
a strong position because..." or "Smith agrees with position X". First person language identified in your writing 
below with <span class="firstperson">red colouring</span>.
</p>
<p class="feedback"><b>Wordiness - </b>Using too many unnecessary words can result in your writing losing clarity, and 
result in it being more difficult to read. In your text below, adjectives and adverbs are <span class="qualifier">
coloured blue</span>. In sentences where there are many blue words, try changing the sentence so that you say the same
thing, but with less words.
</p>
<p><b>Your text:</b></p>
"""

markedup = map(wrapPara,map(markupQualifiers,map(markupFirstPerson,doc)))
html = HTML(comment + ''.join(markedup))

css = HTML("""
<style>
.feedback {
    color: darkslateblue;
}
.firstperson {
    color: red;
}
.qualifier {
    color: blue;
}
</style>

""")

display(css,html)

> ** DISCUSSION **
>
>  - Do you see any problem with this feedback? 
>  - Does text analytics pick all the right words that need changing?
